# Train Linear Classifier on Binary Mental Task EEG Data

In [1]:
from analysis_helpers import *

#experiment title
title = "BCI"

## Get Subject Information

In [2]:
#navigate to subject data
subject_info = get_subject_info(title)
os.chdir(f'../data/{subject_info}')

## Set Parameters

In [3]:
#set parameters
classes = ['hand','word']
classifiers = {'LDA': LinearDiscriminantAnalysis(),'LR':LogisticRegression(),'SVM':SVC()}
transforms_dict= {'CSP': CSP(n_components=n_components, reg=None, log=True, norm_trace=False)}


transforms = [('CSP',transforms['CSP'])]
model = classifiers[model_name]
model_name='LR'
encoding = {classes[0]:0, classes[1]:1}

## Load DataFrame

In [ ]:
#load subject data as df
df = pd.read_csv(os.path.join(os.getcwd(),f'{subject_info}.csv'))
df.head

## Filter and Split Data

In [ ]:
#filter df by class
filter_df = df[df['Category'].str.contains(classes[0]) | df['Category'].str.contains(classes[1])]

#set X and y values
X = filter_df['Feature Vector']
y = [encoding[x] for x in list(filter_df['Category'])]

#split data into train test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20,random_state=1)

## Set up Pipeline

In [ ]:
#create pipeline with transform and model
steps = transforms.append((model_name, model))
clf = Pipeline(steps)
print(len(X_train),len(y_train))

##  Run Cross Validation and Fit Model

In [ ]:
#get cross validation score
scores = cross_val_score(clf, X_train, y_train, cv=k, n_jobs=1)
print('Mean Cross Validation Score: {text}'.format(text=scores.mean()))

#extract common spatial features
X_train, X_test = csp.fit_transform(X_train, y_train), csp.transform(X_test)

#fit to train data and save best model
model.fit(X_train,y_train,verbose=1,max_iter=60)

#score model on validation data
validation_score = model.score(X_test,y_test)
print(validation_score)

## Save Model

In [ ]:
#set save path
save_path = os.path.join(os.getcwd(),f'{model_name}_{classes[0]}_{classes[1]}.sav')
#save model
joblib.dump(model,save_path)